# PyCaret with Synthetic Data for Real-Time Credit Card Fraud detection
\[_In case you’re unable to see the atoti visualizations in GitHub, try viewing the notebook in [nbviewer](https://nbviewer.org/github/atoti/notebooks/blob/master/notebooks/credit-card-fraud-detection/main.ipynb)._]  

__NOTE: As of [PyCaret v2.3.9](https://pycaret.gitbook.io/docs/get-started/installation#dependencies) and [atoti v0.6.5](https://docs.atoti.io/latest/installation.html), there are some dependencies conflict in the version of NumPy used. Hence, the machine learning aspect in this notebook is used for reference purposes only.__ 

[Credit card fraud is on the rise](https://www.forbes.com/sites/billhardekopf/2021/09/17/this-week-in-credit-card-news-huge-increase-in-fraudulent-mobile-apps-a-rosy-holiday-sales-outlook/?sh=385215f866a1) with the increase of online transactions due to COVID-19.  

In this use case, we will attempt a couple of fun things:
- [Generate synthetic data for machine learning](./01-Synthetic-data-generation.ipynb)
- Test out the autoML PyCaret in predicting the fraudulent cases with the [Anomaly Detection](./02-AutoML-PyCaret-anomaly.ipynb) and [Classification module](./03-AutoML-PyCaret-classification.ipynb)
- Integrate the prediction into atoti for real-time credit card reporting

The use case is only for illustrative purpose to show how atoti can be integrated with machine learning to achieve monitoring objective.  

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=creditcard-fraud-detection" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover.png" alt="Try atoti"></a></div>

## 1. Generating Synthetic Data

The __sparkov_data_generation__ package used in this use case is forked from the GitHub repo [Synthetic Credit Card Transaction Generator used in the Sparkov program](https://github.com/namebrandon/Sparkov_Data_Generation).  

Use [01-Synthetic-data-generation.ipynb](./01-Synthetic-data-generation.ipynb) to generate the synthetic data or download the pre-generated data from atoti s3 folder.

In [ ]:
import os
import time
from zipfile import ZipFile

import atoti as tt

# import faker
import pandas as pd
import wget

# from faker import Faker
from IPython.display import clear_output, display
from utils import data_prep

In [ ]:
data_path = "./output"

Uncomment and run the below two cells to download the pre-generated synthetic data.

In [ ]:
# def bar_custom(current, total, width=80):
#     clear_output(wait=True)
#     print("Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total))


# url = "https://s3.eu-west-3.amazonaws.com/data.atoti.io/notebooks/credit-card-fraud/output.zip"
# filename = wget.download(url, bar=bar_custom)

In [ ]:
# # unzipping the file
# with ZipFile("output.zip", "r") as zipObj:
#     # Extract all the contents of zip file in current directory
#     zipObj.extractall()

## 2. Creating atoti cube for data exploration

We load the synthetic data into atoti tables and spin up a multidimensional cube to explore the data.  
To begin, we create an atoti session.

### 2.1 Create atoti session with SessionConfig
Below, we configured the [`SessionConfig`](https://docs.atoti.io/latest/lib/atoti.config.session_config.html#atoti.config.session_config.SessionConfig) with:
- user_content_storage: for persisting dashboards
- port: fix the port number, otherwise a random port will be assigned for the web application

In [ ]:
session = tt.Session(user_content_storage="./content", port=10327)

atoti can load CSV directly into the table. However, we will be performing machine learning on the data set later on in the notebook.  
So, let's read the customers and transactions data into a Pandas DataFrame.

In [ ]:
cust_df = pd.read_csv(
    f"{data_path}/post_process_customer_list.csv", parse_dates=["dob"]
)

txns_df = pd.read_csv(
    f"{data_path}/post_process_transaction_list.csv", parse_dates=["trans_date"]
)

Below are some columns that should be treated as strings even though they are of numerical values. This is so that they will be automatically created as hierarchies instead of measures (Refer to [mode for create_cube](https://docs.atoti.io/latest/lib/atoti.session.html#atoti.session.Session.create_cube)).

In [ ]:
common_col = ["ssn", "cc_num", "acct_num"]
cust_non_num_col = ["ssn", "cc_num", "acct_num", "zip"]
txn_non_num_col = [
    "ssn",
    "cc_num",
    "acct_num",
    "trans_num",
    "unix_time",
    "merchant_id",
    "is_fraud",
    "txn_during_night",
    "trans_weekend",
]
txn_col = [
    "ssn",
    "cc_num",
    "acct_num",
    "profile",
    "trans_num",
    "trans_date",
    "trans_time",
    "unix_time",
    "category",
    "amt",
    "is_fraud",
    "merchant",
    "merchant_id",
    "merch_long",
    "merch_lat",
    "distance_from_cust",
    "txn_during_night",
    "trans_weekend",
]

Let's cast the data types for the columns before loading them into the cube.

In [ ]:
cust_df[cust_non_num_col] = cust_df[cust_non_num_col].astype(str)
txns_df[txn_non_num_col] = txns_df[txn_non_num_col].astype(str)

### 2.2 Load data into atoti table

We can load data into atoti with either:
- read_*: create an atoti table and loads the data
- load_*: load data into an existing atoti table

Below, we will load data for the customers and transactions into atoti tables with [`read_pandas`](https://docs.atoti.io/latest/lib/atoti.session.html#atoti.session.Session.read_pandas).  

#### 2.2.1 Base table - transaction

Transactions is more granular than customers. Since we are following the [snowflake schema](https://en.wikipedia.org/wiki/Snowflake_schema#:~:text=In%20computing%2C%20a%20snowflake%20schema,diagram%20resembles%20a%20snowflake%20shape.&text=The%20principle%20behind%20snowflaking%20is,attributes%20and%20forming%20separate%20tables.), we will choose the transaction table as our base table for cube creation.

In [ ]:
txn_tbl = session.read_pandas(
    txns_df[txn_col],
    table_name="transactions",
    keys=["ssn", "cc_num", "acct_num", "trans_num"],
)

In [ ]:
txn_tbl.head()

In [ ]:
len(txn_tbl)

#### 2.2.2 Referenced table - customer

We can create referenced table before or after cube creation. Referenced tables are join to the cube to [enrich it](https://docs.atoti.io/latest/tutorial/tutorial.html#Enriching-the-cube).

In [ ]:
customer_tbl = session.read_pandas(
    cust_df,
    table_name="customer",
    keys=common_col,
)

In [ ]:
len(customer_tbl)

In [ ]:
customer_tbl.head()

#### 2.2.3 Create empty atoti table - prediction

The PyCaret prediction output 2 columns:
- Label
- Score

With [`atoti.create_table`](https://docs.atoti.io/latest/lib/atoti.session.html#atoti.session.Session.create_table), we can create an empty table. We will need to define the columns, its expected data type as well as the table keys (if any).

In [ ]:
prediction_tbl = session.create_table(
    name="prediction",
    types={
        "trans_num": tt.type.STRING,
        "Label": tt.type.STRING,
        "Score": tt.type.FLOAT,
    },
    keys=["trans_num"],
)

To demonstrate how we can load data into an existing table, we will load the prediction from the LGBM model for July into `prediction_tbl`:

In [ ]:
prediction_tbl.load_csv(f"{data_path}/july_aug_prediction_full_lgbm.csv")

In [ ]:
prediction_tbl.head()

### 2.3 Join tables

We join the referenced table _customer_ to the base table _transaction_. We did not explicity configure the mapping as atoti automatically infers the join based on the common column names.  

We can also join any new tables to the base table after the cube is created.

Refer to the [join examples in tutorial](https://docs.atoti.io/latest/tutorial/tutorial.html#Join) for more details.

In [ ]:
txn_tbl.join(customer_tbl)
txn_tbl.join(prediction_tbl, mapping={"trans_num": "trans_num"})

### 2.4 Create cube

We create the cube with the base table. Cube name `credit_card_trans` is optional.

In [ ]:
cube = session.create_cube(txn_tbl, "credit_card_trans")

#### 2.4.1 View cube schema

Once the cube is created, we can visualize the cube schema. If you [installed atoti via Python package managers such as pip](https://docs.atoti.io/latest/installation.html#python-package), you need to install  [graphviz](https://www.graphviz.org/download/) to do this step.

In [ ]:
cube.schema

#### 2.4.2 Alias cube hierarchies property

We can manipulate the below aspects of the cube. Aliasing the hierarchies property makes it more convenient for use later on.

In [ ]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

### 2.5 Access web aplication

Upon session creation, a web application is launched. However, without any cubes, we can't perform any analysis on the web application.  

Run [`session.link( )`](https://docs.atoti.io/latest/lib/atoti.session.html#atoti.session.Session.link) to retrieve the URL of the web application.

In [ ]:
session.link()

### 2.6 Visualize data in Jupyter notebook

Once the cube is created, we can make use of the atoti JupyterLab extension to [create visualization](https://docs.atoti.io/latest/tutorial/tutorial.html#First-visualization) interactively. Let's take a quick look at the number of fraudulent and non-fraudulent transactions.  

#### 2.6.1 Showing text on charts

At the moment, we can enable text display on the chart by editing the Jupyter notebook's Cell Metadata to add `"isTextVisible": true` under the `widget` node:
<img src="img/showText.png" width="80%"/>   

From the chart below, we can see that the number of fraudulent cases is less than 5%.

In [ ]:
session.visualize("0 - Non-fraudulent, 1 - Fraudulent")

#### 2.6.2 Subplots

Simply add the levels to the vertical or horizontal subplot section in the atoti JupyterLab extension.  

We ought to be careful of those merchants with low transaction volume and high occurrence of fraud, likely a dubious merchant. Though, it's also likely the higher the transaction volume, the more susceptible the merchant is to external fraud.

In [ ]:
session.visualize("Merchants with highest fraudulent occurence on 29th Aug")

Is it coincidence that some merchants have multiple fraudulent transactions within a day?

### 2.7 Creating measures 

Measures in atoti is computed on the fly as we query the cube. When a cube is created in [auto mode](https://docs.atoti.io/latest/lib/atoti.session.html#atoti.session.Session.create_cube), a ".SUM" and a ".MEAN" measures are created for numerical columns of the base table. Measures are not created for the referenced table.

We can make use of the `cube.measures` property to define new measures. 

#### 2.7.1 Compute measures for referenced table 

Customer table is a referenced table. Hence, we have to create the necesary measures manually.  
We can do so with [`atoti.single_value`](https://docs.atoti.io/latest/lib/atoti.html#atoti.agg.single_value) which will return the measure value when queried at levels corresponding to the keys of column's table. Alternatively, we can perform aggregation on the table column, which is what we will do in this case. 

Below, we create the measures for customer's longitude, latitude and age.

In [ ]:
m["cust_long"] = tt.agg.mean(customer_tbl["long"])
m["cust_lat"] = tt.agg.mean(customer_tbl["lat"])
m["cust_age"] = tt.agg.mean(customer_tbl["age"])

In [ ]:
session.visualize(
    "Number of transactions by customer in 1 day on 29th Aug - (x-axis: longitude, y-axis: latitude)"
)

Similarly, we will create the measure `score` for the prediction table with the [summation aggregator](https://docs.atoti.io/latest/lib/atoti.agg.html#atoti.agg.sum).

In [ ]:
m["score"] = tt.agg.sum(prediction_tbl["Score"])
m["score"].formatter = "DOUBLE[#,###.000]"

#### 2.7.2 Count distinct values

We should be seeing increase in customers over the months. Let's create some measures to see if this increase affects the number of fraudulent transactions.

We count the number of active customers in order to see if the increase in transactions is also due to increase in customer. We can use the atoti function [`count_distinct`](https://docs.atoti.io/latest/lib/atoti.agg.html#atoti.agg.count_distinct) on the table column that stores the identified for customer, `ssn` (Social Security Number) in this case.

In [ ]:
m["nb_active_customers"] = tt.agg.count_distinct(customer_tbl["ssn"])

#### 2.7.3 Rolling windows 

We could have achieve the below with Pandas:

```
def get_spending_features(txn, windows_size=[1, 7, 30]):
    # Setting trans_date as index for rolling function
    txn.index = txn.trans_date

    for size in windows_size:
        # compute the total transaction amount and the number of transactions during the window
        rolling_tx_amt = txn["amt"].rolling(f"{size}D").sum()
        roll_tx_cnt = txn["amt"].rolling(f"{size}D").count()

        # compute the average transaction amount
        avg_trans_amt = rolling_tx_amt / roll_tx_cnt

        # create as new columns
        txn[f"nb_trans_{size}days"] = list(roll_tx_cnt)
        txn[f"cust_avg_amt_{size}days"] = list(avg_trans_amt)

    # Reindex according to transaction IDs
    txn.index = txn.trans_num

    # And return the dataframe with the new features
    return txn
    
cc_txn_df = cc_txn_df.groupby("acct_num").apply(
    lambda x: get_spending_features(x, windows_size=[1, 7, 30])
)
```

However, this preprocessed information is not dynamic enough. We will need to get the cumulative value on the fly as we perform real-time fraud detection.  
Hence, we will create the cumulative number of transactions for windows of 1, 7 and 30 days in atoti.  

Be mindful that previously, we applied the `DESCENDING` comparator on the `trans_date`. Hence, we should be looking at the _next members_ instead of _previous members_.

In [ ]:
m["nb_txns_1_days"] = tt.agg.sum(
    m["contributors.COUNT"], scope=tt.OriginScope(l["trans_date"])
)

m["nb_txns_7_days"] = tt.agg.sum(
    m["contributors.COUNT"],
    scope=tt.CumulativeScope(level=l["trans_date"], window=("-P7D", None), dense=True),
)
m["nb_txns_30_days"] = tt.agg.sum(
    m["contributors.COUNT"],
    scope=tt.CumulativeScope(level=l["trans_date"], window=("-P30D", None), dense=True),
)

We will also compute the average transaction amount for windows of 1, 7 and 30 days.

In [ ]:
m["avg_txns_amt_1_days"] = tt.agg.mean(
    m["amt.MEAN"], scope=tt.OriginScope(l["trans_date"])
)

m["avg_txns_amt_7_days"] = tt.agg.mean(
    m["amt.MEAN"],
    scope=tt.CumulativeScope(level=l["trans_date"], window=("-P7D", None), dense=True),
)
m["avg_txns_amt_30_days"] = tt.agg.mean(
    m["amt.MEAN"],
    scope=tt.CumulativeScope(level=l["trans_date"], window=("-P30D", None), dense=True),
)

In [ ]:
session.visualize("Cumulative values for a customer account - SSN (862-79-0516)")

In [ ]:
session.link()

##### 2.7.3.1 Cube querying with filter

Instead of visualization, we can query the cube. It returns the results in a __Pandas DataFrame__.  
Below, we apply the filters on the `ssn` and `trans_date` to get its filtered values. This will come in handy as we query the cumulative values for machine learning later on.

In [ ]:
cube.query(
    m["nb_txns_1_days"],
    m["nb_txns_7_days"],
    m["nb_txns_30_days"],
    levels=[l["trans_date"]],
    filter=((l["ssn"] == "862-79-0516") & (l["trans_date"] == "2021-07-12")),
)

#### 2.7.4 Measure Filter

In the previous section, we saw how we can apply filter on the visualization or query.  
Alternatively, we can use the [`filter`](https://docs.atoti.io/latest/lib/atoti.html#atoti.filter) to apply filter on a measure. In this case, we want the count where the transactions are marked as fraudulent.

In [ ]:
m["nb_fraudulent_txns"] = tt.filter(m["contributors.COUNT"], l["is_fraud"] == "1")

#### 2.7.5 Chain measures 

We can define a measure on top of other measures. For instance:

In [ ]:
m["%_fraud_txns"] = m["nb_fraudulent_txns"] / m["contributors.COUNT"]
m["%_fraud_txns"].formatter = "DOUBLE[#.000%]"

Any updates to `m["nb_fraudulent_txns"]` will have cascading effect on `m["%_fraud_txns"]`.

Let's sum up the total amount loss to fraud and see the percentage loss.

In [ ]:
m["fraudulent_txn_amt"] = tt.agg.sum(
    tt.where((l["is_fraud"] == "1"), m["amt.SUM"]),
    scope=tt.OriginScope(l["trans_date"], l["is_fraud"]),
)

m["%_loss_to_fraud"] = tt.where(
    m["fraudulent_txn_amt"] == 0, 0, m["fraudulent_txn_amt"] / m["amt.SUM"]
)
m["%_loss_to_fraud"].formatter = "DOUBLE[#.000%]"

We can now see which are the merchants with more than 50% fraudulent transactions. There are five merchants with 100% fraudulent transactions.  

##### Tips: Color coding table with MDX
At the moment, we can manipulate the MDX from the Jupyter notebook's Cell Metadata to color the fonts conditionally:
```
WITH  Member [Measures].[%_fraud_txns (1)] AS [Measures].[%_fraud_txns], FORE_COLOR = CASE WHEN [Measures].[%_fraud_txns (1)] > 0.25 THEN rgb(245, 66, 66) END, format_string=\"##.000%\", CAPTION = [Measures].[%_fraud_txns].MEMBER_CAPTION
```

In [ ]:
session.visualize("Merchants with more than 25% percentage fraudulent transactions")

#### 2.7.6 Different ways to create same measure

We can apply the below filter to get the non-fraudulent transactions.   
`m["nb_normal_txns"] = tt.filter(m["contributors.COUNT"], l["is_fraud"] == "0")`  

Alternatively, we can minus the `nb_fraudulent_txns` from the total count. However, any updates to `m["nb_fraudulent_txns"]` will have cascading effect on `m["nb_normal_txms"]`.

In [ ]:
m["nb_normal_txns"] = m["contributors.COUNT"] - m["nb_fraudulent_txns"]

The measures will be available for visualization immediately in both Jupyter notebook and the web application.  

There is a gradual increase in the number of customer over time and the number of fraudulent transactions also seems to increase. However, there is a drop in the number of fraudulent transactions towards the end of September. The overall number of transactions during the same period also fell.

In [ ]:
session.visualize("Fraud trend")

### 2.8 Drillthrough to view underlying data 

Right-click on the `nb_fraudulent_txns` in the chart on 29th Aug (the day with the highest number of fraud occurrence). A context menu will be displayed.  
Select "Drillthrough on 2021-08-29" to look at the underlying transactions.
<img src="img/drillthrough.png" width="80%"/>   

A new cell displaying the drillthrough widget will be displayed. Use the atoti JupyterLab extension on the left to select the features to review.

In [ ]:
session.visualize("Drillthrough on 29th Aug")

### 2.9 Manipulating hierarchies

#### 2.9.1 Create date hierarchy from date column

To visualize yearly or monthly data based on the transaction dates, we can create a new hierarchy as shown below.  
We can also create weekly or quarterly hierarchy if needed. Refer to [atoti documentation](https://docs.atoti.io/latest/lib/atoti.experimental.html#atoti.experimental.create_date_hierarchy) for more information.  

The [date pattern](https://docs.oracle.com/en/java/javase/15/docs/api/java.base/java/time/format/DateTimeFormatter.html#patterns) used below _creates levels of type String_,

In [ ]:
cube.create_date_hierarchy(
    "Transaction Dates",
    column=txn_tbl["trans_date"],
    levels={"Year": "yyyy", "Month": "MM", "Day": "dd"},
)

Using the visualization type __KPI__, we can show the difference in amount loss to fraud between the different months.

In [ ]:
session.visualize("Amount loss to fraud comparison in last 2 months")

There is an increasing trend in the amount lost to fraud, having a fifth of the transaction amount lost to fraud in August.

In [ ]:
session.visualize("Amount loss to fraud")

### 2.10 Create table for classification 

We can always add additional tables to the cube without having to restart the session. Let's create another atoti table to group the customers' age. 

In [ ]:
age_groups_store = session.read_pandas(
    pd.DataFrame(
        data=[("0-25Y", i) for i in range(25)]
        + [("25Y - 40Y", i) for i in range(25, 40)]
        + [("40Y - 55Y", i) for i in range(40, 55)]
        + [("55Y+", i) for i in range(55, 100)],
        columns=["age group", "age"],
    ),
    keys=["age"],
    table_name="Age_Groups",
)

customer_tbl.join(age_groups_store)

cube.schema

We can now tell that the youngest and oldest tier are more susceptible to fraud.

In [ ]:
session.visualize("% loss to fraud by age group")

## 3. Integrate prediction from AutoML (PyCaret) into atoti

Using [PyCaret](https://pycaret.org/), we trained and tested 3 months worth of synthetic credit card transactions.  
The trained models are being saved under the __models__ folder.  

### 3.1 Create atoti KPI based on the predictions

#### 3.1.1 Create similar KPI as actual fraud for predicted fraud

Instead of using `is_fraud` which indicates true fraud cases, we will use `Label` to create similar KPIs.

In [ ]:
m["nb_predicted_fraud_txns"] = tt.filter(m["contributors.COUNT"], l["Label"] == "1")
m["%_predicted_fraud_txns"] = m["nb_predicted_fraud_txns"] / m["contributors.COUNT"]

m["%_predicted_fraud_txns"].formatter = "DOUBLE[#.000%]"

In [ ]:
m["predicted_fraud_txn_amt"] = tt.agg.sum(
    tt.where((l["Label"] == "1"), m["amt.SUM"]),
    scope=tt.OriginScope(l["trans_date"], l["is_fraud"], l["Label"]),
)
m["%_predicted_loss_to_fraud"] = tt.where(
    m["predicted_fraud_txn_amt"] == 0, 0, m["predicted_fraud_txn_amt"] / m["amt.SUM"]
)

m["%_predicted_loss_to_fraud"].formatter = "DOUBLE[#.000%]"

In [ ]:
m["diff_with_actual_txn_amt"] = m["fraudulent_txn_amt"] - m["predicted_fraud_txn_amt"]
m["diff_with_actual_nb_fraud_txn"] = (
    m["nb_fraudulent_txns"] - m["nb_predicted_fraud_txns"]
)

Since we have the prediction for July, let's visualize this value against the actual fraud. But we haven't take the recall and precision into account.

In [ ]:
session.visualize("Difference between actual and predicted")

#### 3.1.2 Compute recall and precision for prediction

We can plot the confusion matrix with the actual frauds `is_fraud` on rows and predicted fraud `Label` on columns.  
This shows that out of the 10,305 fraudulent transactions in July and Aug, we predicted 8,757 of them correctly.

In [ ]:
session.visualize("Confusion matrix")

In fact, we can also compute the recall and precision for the prediction with atoti:

In [ ]:
m["true_positive"] = tt.agg.sum(
    tt.where((l["Label"] == "1") & (l["is_fraud"] == "1"), m["contributors.COUNT"]),
    scope=tt.OriginScope(l["trans_date"], l["is_fraud"], l["Label"]),
)

m["true_negative"] = tt.agg.sum(
    tt.where((l["Label"] == "0") & (l["is_fraud"] == "0"), m["contributors.COUNT"]),
    scope=tt.OriginScope(l["trans_date"], l["is_fraud"], l["Label"]),
)

m["false_positive"] = tt.agg.sum(
    tt.where((l["Label"] == "1") & (l["is_fraud"] == "0"), m["contributors.COUNT"]),
    scope=tt.OriginScope(l["trans_date"], l["is_fraud"], l["Label"]),
)

m["false_negative"] = tt.agg.sum(
    tt.where((l["Label"] == "0") & (l["is_fraud"] == "1"), m["contributors.COUNT"]),
    scope=tt.OriginScope(l["trans_date"], l["is_fraud"], l["Label"]),
)

m["accuracy"] = (m["true_positive"] + m["true_negative"]) / m["contributors.COUNT"]
m["precision"] = m["true_positive"] / (m["true_positive"] + m["false_positive"])
m["recall"] = m["true_positive"] / (m["true_positive"] + m["false_negative"])
m["f1 score"] = 2 * (m["precision"] * m["recall"]) / (m["precision"] + m["recall"])

From the below table, we can see that the accuracy rate of the Light Gradient Boosting Machine has accuracy of 99% which is very high.

In [ ]:
session.visualize("Performance of the LGBM model against July prediction")

Let's look at the performance from the monetary perspective.

In [ ]:
m["true_positive_amt"] = tt.agg.sum(
    tt.where((l["Label"] == "1") & (l["is_fraud"] == "1"), m["amt.SUM"]),
    scope=tt.OriginScope(l["trans_date"], l["is_fraud"], l["Label"]),
)

m["false_positive_amt"] = tt.agg.sum(
    tt.where((l["Label"] == "1") & (l["is_fraud"] == "0"), m["amt.SUM"]),
    scope=tt.OriginScope(l["trans_date"], l["is_fraud"], l["Label"]),
)

m["false_negative_amt"] = tt.agg.sum(
    tt.where((l["Label"] == "0") & (l["is_fraud"] == "1"), m["amt.SUM"]),
    scope=tt.OriginScope(l["trans_date"], l["is_fraud"], l["Label"]),
)

m["% amt loss prevented"] = m["true_positive_amt"] / m["fraudulent_txn_amt"]
m["% amt loss prevented"].formatter = "DOUBLE[##.00%]"

`true_positive_amt`: if preventive actions were taken against these transactions, __more than a million dollars will be saved__ from fraudulent activities.  
`false_positive_amt`: even though falsely predicted, it's better to be safe than sorry.  
`false_negative_amt`: fraudulent transactions missed by the machine learning and resulted in actual loss.  
`% amt loss prevented`: prevents __more than 90% of the actual fraudulent transaction amount__ from actual loss.

In [ ]:
session.visualize("Transaction amount saved/lost to fraud after preventive actions")

#### 3.1.3 Place measures in folder  

Since atoti is a Python library, we can iterate through the measures to put them in folders conditionally.

In [ ]:
for measure_name in m.keys():
    if "predicted" in measure_name:
        m[measure_name].folder = "Prediction"

In [ ]:
for measure in [
    m["true_positive"],
    m["true_negative"],
    m["false_positive"],
    m["false_negative"],
    m["accuracy"],
    m["precision"],
    m["recall"],
    m["true_positive_amt"],
    m["false_positive_amt"],
    m["false_negative_amt"],
    m["% amt loss prevented"],
]:
    measure.folder = "Evaluation"

for measure in [
    m["accuracy"],
    m["precision"],
    m["recall"],
]:
    measure.formatter = "DOUBLE[##.0000%]"

### 3.2 Model comparisions with atoti source simulation  

PyCaret offers 6 different types of module, of which 2 modules are used for credit card fraud detection in most of the articles online:
- Classification 
- Anomaly detection  

Upon deciding on the type of module to use, we also have to consider the type of models we want.  
Also, which are the features that we want to include?

That's a lot of choices and which is the best? Since PyCaret is easy to use, we can easily setup my modules, create some models and run our prediction.  

Let's load them into atoti to have a side-by-side comparison. Below, we have the prediction results from the unseen dataset for which, some includes the cumulative features while others exclude them.  

In [ ]:
prediction_tbl.scenarios["Class - Light Gradient Boosting Machine (Non-Cum)"].load_csv(
    f"{data_path}/july_aug_prediction_lgbm.csv"
)

In [ ]:
prediction_tbl.scenarios["Class - Decision Tree Classifier (Cum)"].load_csv(
    f"{data_path}/july_aug_prediction_full_dt.csv"
)

In [ ]:
prediction_tbl.scenarios["Class - Decision Tree Classifier (Non-cum)"].load_csv(
    f"{data_path}/july_aug_prediction_dt.csv"
)

In [ ]:
prediction_tbl.scenarios["Anomaly Detection - iForest (Cum)"].load_csv(
    f"{data_path}/july_aug_anomalies_full_iforest.csv"
)

In [ ]:
prediction_tbl.scenarios["Anomaly Detection - iForest (Non-cum)"].load_csv(
    f"{data_path}/july_aug_anomalies_iforest.csv"
)

In [ ]:
session.visualize("Confusion matrix")

It seems that the classification module has better performance than the anomaly detection method. The LGBM model has better performance when we include the cumulative number of transactions and average transaction amount as features.  

In [ ]:
session.visualize(
    "Performance comparison for models against July and August prediction"
)

In [ ]:
session.visualize("Amount of money saved by each model")

We can see the difference in the potential amount of money that can be alerted and saved from loss due to fraud, based on the prediction by each model.  
Both LGBM and Decision Tree perform almost equally well.  

It would be interesting to look at the fraudulent transactions detected by the anomaly detection module and not by the classification module. It could highlight a rare instance of fraud case overlooked by supervised learning.

In [ ]:
session.visualize(
    "Anomaly detected on 27th Aug but not identified as fraud by classification"
)

Access the below link to access the model performance dashboard.

In [ ]:
session.link(path="/#/dashboard/cd7")

### 3.3 Launch real-time dashboard 

__NOTE: We have disabled the integration of real-time prediction into atoti due to the dependency conflict between atoti and PyCaret.  
Feel free to enable the prediction once the conflict is resolved.__

In [ ]:
disable_prediction = True

After we have created all the KPIs, we can design our real-time dashboard. Access the saved dashboard from the link below:

In [ ]:
session.link(path="/#/dashboard/4c8")

### 3.4 Load saved ML model

As we can see from the previous section, the Light Gradient Boosting Machine (LGBM) model with full features has the best performance. We will load its saved model for real-time fraud detection simulation below.

Refer to [03-AutoML-PyCaret-classification.ipynb](./03-AutoML-PyCaret-classification.ipynb) for the model evaluation.

In [ ]:
if not disable_prediction:
    from pycaret.classification import *

    model_path = "./models/"
    saved_final_lgbm = load_model(f"{model_path}Final LGBM Model 20211130")

### 3.5 Perform prediction on real-time data

We will iterate through the September transactions that was put aside from the ML modeling previously.  
For each iteration, we will perform the prediction and load the results back into atoti, along with the transactions.  

We can monitor the various KPI for fraud investigation.  

#### 3.5.1 Features to be passed to model

In [ ]:
feature_list = [
    "category",
    "amt",
    "is_fraud",
    "merchant_id",
    "gender",
    "street",
    "city",
    "state",
    "zip",
    "city_pop",
    "job",
    "age",
    "distance_from_cust",
    "txn_during_night",
    "trans_weekend",
    "nb_txns_1_days",
    "avg_txns_amt_1_days",
    "nb_txns_7_days",
    "avg_txns_amt_7_days",
    "nb_txns_30_days",
    "avg_txns_amt_30_days",
]

#### 3.5.2 Throttling transaction data reading

Using the [`chunksize`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) from Pandas, we can choose the number of rows to read per iteration. We have data for the month of September, so we can adjust the chunksize for longer or shorter data loading duration (to achieve real-time demo effect).

In [ ]:
if not disable_prediction:
    new_txns_df = pd.read_csv(
        # f"{data_path}/realtime_txn.csv", # uncomment for more data to have a longer streaming time
        f"{data_path}/realtime_txn_subset.csv",  # using smaller dataset to facilitate testing in GIT
        parse_dates=["trans_date"],
        chunksize=1000,
    )

#### 3.5.3 Custom prediction function

The function below does the following: 
1. computes the features required as per [03-AutoML-PyCaret-classification.ipynb](./03-AutoML-PyCaret-classification.ipynb)
2. invoke the [`predict_model`](https://pycaret.org/predict-model/) function on the features data with the selected model
3. load the prediction into the prediction table created earlier on (2.2.3)
4. load transactions into the transaction table

In [ ]:
def run_prediction(_txn_df, model, debug=False):

    print(".", end=" ")
    _txn_df[common_col] = _txn_df[common_col].astype(str)

    # perform data transformation
    if debug == True:
        print("1 ... Generating additional features for transaction")

    features_df = data_prep.compute_features(cust_df, _txn_df, ignore_features=True)
    features_df.reset_index(drop=True, inplace=True)

    # load transaction into atoti transaction table
    realtime_txn_df = features_df[txns_df.columns].copy()
    realtime_txn_df[txn_non_num_col] = realtime_txn_df[txn_non_num_col].astype(str)

    if debug == True:
        print("2 ... Loading transactions into atoti")

    txn_tbl.load_pandas(realtime_txn_df)
    time.sleep(1)

    if debug == True:
        print("2 ... Getting cumulative values from atoti cube")

    cc_list = list(set(realtime_txn_df["cc_num"].tolist()))
    trans_date = list(
        set(realtime_txn_df["trans_date"].dt.strftime("%Y-%m-%d").tolist())
    )

    # we can only query after transactions have been loaded into the cube
    cum_df = cube.query(
        m["nb_txns_1_days"],
        m["nb_txns_7_days"],
        m["nb_txns_30_days"],
        m["avg_txns_amt_1_days"],
        m["avg_txns_amt_7_days"],
        m["avg_txns_amt_30_days"],
        levels=[l["cc_num"], l["acct_num"], l["trans_date"]],
        condition=((l["cc_num"].isin(*cc_list)) & (l["trans_date"].isin(*trans_date))),
    )

    cum_df.reset_index(inplace=True)
    features_df = features_df.merge(cum_df)

    # perform prediction
    if debug == True:
        print("4 ... Performing machine learning prediction")

    features = features_df[feature_list]
    model_prediction = predict_model(model, data=features)

    # merge prediction results back to original dataset
    features_df = features_df.merge(model_prediction)

    features_df[["trans_num", "Label"]] = features_df[["trans_num", "Label"]].astype(
        str
    )

    if debug == True:
        print("----- ", features_df[prediction_tbl.columns].head(3))
        print(".. Loading predictions into atoti")

    # load prediction into atoti prediction table
    prediction_tbl.load_pandas(features_df[prediction_tbl.columns])

#### 3.5.4 Iterate dataset to simulate prediction of incoming data stream

We perform fraud prediction with the _Light Gradient Boosting Machine_ model which has the best AUC.  

In [ ]:
if not disable_prediction:

    for chunk in new_txns_df:
        run_prediction(chunk, saved_final_lgbm)

While PyCaret allows us to compare multiple machine learning models in one go, atoti translate the predictions and allows us to compare downstream business metrics side-by-side. The ability to compute the different KPIs on the fly reduce the turnaround time needed to discover a potential fraud. Hence, leveraging atoti, we can choose the most efficient machine learning model based on business KPI.  

Have a try with other models.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=creditcard-fraud-detection" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover-try.png" alt="Try atoti"></a></div>